In [1]:
import os
import sys

In [2]:
spark_path = "E:/spark"

In [3]:
os.environ['SPARK_HOME'] = spark_path

In [4]:
os.environ['HADOOP_HOME'] = spark_path

In [5]:
os.environ['PYSPARK_PYTHON'] = sys.executable

In [6]:
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

In [7]:
from pyspark import SparkContext
from pyspark import SparkConf

In [8]:
conf = SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.cores.max", "4")

sc = SparkContext("local", conf=conf)

In [9]:
print sc

In [10]:
sc

In [11]:
flightsPath="file:///E:/BIG DATA/FlightsData/flights.csv"

In [28]:
flights=sc.textFile(flightsPath)

In [29]:
flights

file:///E:/BIG DATA/FlightsData/flights.csv MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0

In [30]:
flights.count()

476881

In [15]:
fligths.take(10)

[u'2014-04-01,19805,1,JFK,LAX,0854,-6.00,1217,2.00,355.00,2475.00',
 u'2014-04-01,19805,2,LAX,JFK,0944,14.00,1736,-29.00,269.00,2475.00',
 u'2014-04-01,19805,3,JFK,LAX,1224,-6.00,1614,39.00,371.00,2475.00',
 u'2014-04-01,19805,4,LAX,JFK,1240,25.00,2028,-27.00,264.00,2475.00',
 u'2014-04-01,19805,5,DFW,HNL,1300,-5.00,1650,15.00,510.00,3784.00',
 u'2014-04-01,19805,6,OGG,DFW,1901,126.00,0640,95.00,385.00,3711.00',
 u'2014-04-01,19805,7,DFW,OGG,1410,125.00,1743,138.00,497.00,3711.00',
 u'2014-04-01,19805,8,HNL,DFW,1659,4.00,0458,-22.00,398.00,3784.00',
 u'2014-04-01,19805,9,JFK,LAX,0648,-7.00,1029,19.00,365.00,2475.00',
 u'2014-04-01,19805,10,LAX,JFK,2156,21.00,0556,1.00,265.00,2475.00']

In [32]:
flightsParsed=fligths.map(lambda x: x.split(','))

In [33]:
flightsParsed.first()

[u'2014-04-01',
 u'19805',
 u'1',
 u'JFK',
 u'LAX',
 u'0854',
 u'-6.00',
 u'1217',
 u'2.00',
 u'355.00',
 u'2475.00']

In [34]:
from datetime import datetime
from collections import namedtuple

fields = ('date','airline','flightnum','origin','dest','dep','dep_delay','arv','arv_delay','airtime','distance')

Flight = namedtuple('Flight',fields,verbose=True)

DATE_FMT="%Y-%m-%d"

TIME_FMT ="%H%M"

def parse(row):
    row[0] = datetime.strptime(row[0], DATE_FMT).date()
    row[5] = datetime.strptime(row[5], TIME_FMT).time()
    row[6] = float(row[6])
    row[7] = datetime.strptime(row[7], TIME_FMT).time()
    row[8] = float(row[8])
    row[9] = float(row[9])
    row[10]= float(row[10])
    return Flight(*row[:11])

class Flight(tuple):
    'Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'

    __slots__ = ()

    _fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

    def __new__(_cls, date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance):
        'Create new instance of Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'
        return _tuple.__new__(_cls, (date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Flight object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 11:
            raise TypeError('Expected 11 arguments, got %d' % len(result))
        return result

    def __repr__(self):
        'Return a nicely

In [35]:
flightsParsed=flights.map(lambda x: x.split(',')).map(parse)

In [36]:
flightsParsed.first()

Flight(date=datetime.date(2014, 4, 1), airline=u'19805', flightnum=u'1', origin=u'JFK', dest=u'LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [37]:
flightsParsed.map(lambda x:x.distance)

PythonRDD[12] at RDD at PythonRDD.scala:48

In [38]:
totalDistance=flightsParsed.map(lambda x:x.distance).reduce(lambda x,y:x+y)

In [39]:
avgDistance=totalDistance/flightsParsed.count()

In [40]:
print avgDistance

794.858501387


In [56]:
fligthsParsed.filter(lambda x:x.dep_delay>0).count()/float(flightsParsed.count())

0.3753871510922012

In [57]:
flightsParsed.persist()

PythonRDD[17] at RDD at PythonRDD.scala:48

In [60]:
sumCount=flightsParsed.map(lambda x:x.dep_delay).aggregate((0,0),
                                                           (lambda some, value: (some[0] + value, some[1]+1)),
                                                           (lambda some1,some2: (some1[0]+some2[0],some1[1]+some2[1])))

In [61]:
print sumCount

(3964730.0, 476881)


In [62]:
sumCount[0]/float(sumCount[1])

8.313877046894298

In [63]:
flightsParsed.map(lambda x: int(x.dep_delay/60)).countByValue()

defaultdict(int,
            {0: 452963,
             1: 16016,
             2: 4893,
             3: 1729,
             4: 701,
             5: 249,
             6: 113,
             7: 66,
             8: 43,
             9: 26,
             10: 15,
             11: 12,
             12: 9,
             13: 15,
             14: 13,
             15: 4,
             17: 2,
             20: 4,
             21: 3,
             24: 3,
             25: 1,
             28: 1})